In [ ]:
%pylab inline
#Imports from standard packages
import pickle
from scipy.stats import spearmanr,linregress
from pandas import read_csv

#import from our moulin+subglacial conduit model
from subglacial_channel.simple_conduit_moulin_model import simple_sim


### Comparison between observed oscillations and example simulations

In [ ]:
#Read in moulin water level data
jeme_mou = read_csv('data/jeme.csv', index_col=0, parse_dates = True)
radi_mou = read_csv('data/radi.csv', index_col=0, parse_dates = True)

In [ ]:
#Create Figure 2 from manuscript

figure(figsize=(9,4))
subplot(121)

jeme_mou.water_level_above_bed_fob.plot()
radi_mou.water_level_above_bed_fob.plot()
ylim([0.5,0.85])
ylabel('Observed moulin head ($h^*$)')
xlabel('')
ax = gca()
text(0.035,0.93, 'a', fontsize=14, transform=ax.transAxes)

legend(['Low Camp','High Camp'])

print("mean jeme =",mean(jeme_mou['8-9-2017':'8-16-2017']))
print("max jeme =",max(jeme_mou['8-9-2017':'8-16-2017']))
print("min jeme =",min(jeme_mou['8-9-2017':'8-16-2017']))

print("mean radi =",mean(radi_mou['8-3-2017':'8-7-2017']))
print("max radi =",max(radi_mou['8-3-2017':'8-7-2017']))
print("min radi =",min(radi_mou['8-3-2017':'8-7-2017']))


#figure()
sim_params = {'R_func':'sin',
              'A':3e-24, 
              'overflow':True, 
              'f':0.075,
              'A_R':100,
              'R_mean':0.3,
              'R_min':0.05,
              'h_term':-150.,
              'L':25000.
             }
low_sim = simple_sim(camp='low', params=sim_params)
low_sim.run(run_params={'method':'LSODA','rtol':1e-8, 'atol':1e-8, 't_f':100})

osc_range = max(low_sim.sol.y[0][-500:]) - min(low_sim.sol.y[0][-500:])
print('Low camp osc_range=',str(osc_range))
ex_T1s=[low_sim.T1]
ex_T2s=[low_sim.T2]
ex_R_ratios=[(2*sim_params['R_mean']-sim_params['R_min'])/sim_params['R_min']]
ex_fs = [low_sim.tau_res/low_sim.params['R_period']]
ex_osc_ranges = [osc_range]



sim_params = {'R_func':'sin',
              'A':3e-24, 
              'overflow':True, 
              'f':0.075,
              'A_R':1500,
              'R_mean':3.,
              'R_min':2.,
              'h_term':200.,
              'L':15000.
             }
high_sim = simple_sim(camp='high', params=sim_params)
high_sim.run(run_params={'t_f':200 ,'method':'LSODA','rtol':1e-8, 'atol':1e-8})

osc_range = max(high_sim.sol.y[0][-500:]) - min(high_sim.sol.y[0][-500:])
print('High camp osc_range=',str(osc_range))
ex_T1s.append(high_sim.T1)
ex_T2s.append(high_sim.T2)
ex_R_ratios.append((2*sim_params['R_mean']-sim_params['R_min'])/sim_params['R_min'])
ex_fs.append(high_sim.tau_res/high_sim.params['R_period'])
ex_osc_ranges.append(osc_range)

subplot(122)
secs_per_day = 60*60*24
low_t = low_sim.sol.t*low_sim.tau_res/secs_per_day - 130
plot(low_t, low_sim.sol.y[0])
high_t = high_sim.sol.t*high_sim.tau_res/secs_per_day - 280
plot(high_t, high_sim.sol.y[0])

legend(['Low Camp Model','High Camp Model'])

xlim([0,10])
ylim([0.5,0.85])
ylabel('Modeled moulin head ($h^*$)')
xlabel('Time (days)')
ax = gca()
ax.xaxis.set_ticks([0,1,2,3,4,5,6,7,8,9,10])
text(0.035,0.93, 'b', fontsize=14, transform=ax.transAxes)
tight_layout()
savefig('Moulin-head-variations.png',dpi=300)


### Exploring range of dimensionless params


In [ ]:
#Dimensionless param ranges for high camp
fs=[0.01,0.1,1,10,100]#DW-friction factor
R_means = [1.,5.,10.]
R_factors = [0.1,0.33,0.66]
A_glens = [4e-25, 2e-24, 1e-23]
T1s = []
T2s = []
A_Rs=[10.,100.,200.,500,1000,2000] #moulin surface area (m^2)
for A_glen in A_glens:
    for A_R in A_Rs:
        for f in fs:
            for R in R_means:
                for R_factor in R_factors:
                    params = {}
                    params['A'] = A_glen
                    params['A_R'] = A_R
                    params['f'] = f
                    params['R_func'] = 'sin'
                    params['R_mean'] = R
                    this_sim = simple_sim(params, camp='high')
                    T1s.append(this_sim.T1)
                    T2s.append(this_sim.T2)
hist(log10(T1s))
xlabel('T_melt')
ylabel('Frequency')
figure()
hist(log10(T2s))
xlabel('T_creep')
ylabel('Frequency')

print("Max T1s = ",str(max(log10(T1s))))
print("Min T1s = ",str(min(log10(T1s))))
print("Max T2s = ",str(max(log10(T2s))))
print("Min T2s = ",str(min(log10(T2s))))

In [ ]:
#Dimensionless param ranges for high camp
fs=[0.01,0.1,1,10,100]#DW-friction factor
R_means = [1.,5.,10.]
R_factors = [0.1,0.33,0.66]
A_glens = [4e-25, 2e-24, 1e-23]
T1s = []
T2s = []
tau_stors = []
A_Rs=[10.,100.,200.,500,1000,2000] #moulin surface area (m^2)
for A_glen in A_glens:
    for A_R in A_Rs:
        for f in fs:
            for R in R_means:
                for R_factor in R_factors:
                    params = {}
                    params['A'] = A_glen
                    params['A_R'] = A_R
                    params['f'] = f
                    params['R_func'] = 'sin'
                    params['R_mean'] = R
                    this_sim = simple_sim(params, camp='low')
                    T1s.append(this_sim.T1)
                    T2s.append(this_sim.T2)
                    tau_stors.append(this_sim.tau_res)
hist(log10(T1s))
xlabel('T_melt')
ylabel('Frequency')
figure()
hist(log10(T2s))
xlabel('T_creep')
ylabel('Frequency')
figure()
hist(log10(tau_stors))
xlabel(r'$\tau_{\rm res}$')
ylabel('Frequency')


print("Max T1s = ",str(max(log10(T1s))))
print("Min T1s = ",str(min(log10(T1s))))
print("Max T2s = ",str(max(log10(T2s))))
print("Min T2s = ",str(min(log10(T2s))))
print("Max tau_res = ",str(max((tau_stors))))
print("Min tau_res = ",str(min((tau_stors))))


In [ ]:
#Constants
rho_i = 900.
rho_w = 1000.
g = 9.8

A_R_max = max(A_Rs)
A_R_min = min(A_Rs)
Z_max = 700.
Z_min = 500.
R_mean_max = max(R_means)
R_mean_min = min(R_means)

#R ratio 
print('R_ratio')
print('max=',1.95/0.05)
print('min=',1.2/0.8)

#R_period
print('R_period')
secs_per_day = 60.*60.*24.
P = secs_per_day
tau_stor_max = 10**6.
print('tau_stor_max='+str(tau_stor_max))
tau_stor_min = 10**3.5
print('tau_stor_min='+str(tau_stor_min))
print('R_period_max = '+str(P/tau_stor_min), '   f_min=',str(tau_stor_min/P))
print('R_period_min = '+str(P/tau_stor_max), '   f_max=',str(tau_stor_max/P))

In [ ]:
#use bracketing values of f=0.03 and f=12
print('R_period_max =', 1./0.03)
print('R_period_min = ', 1./12)


### Random simulation set to explore controls on diurnal amplitude

In [ ]:
#Determined from histograms of parameter searches near camps
log_T1_min = -4.
log_T1_max = 0.5

log_T2_min = -3.
log_T2_max = 2.5

R_min_min = 0.05
R_min_max = 0.8
R_period_min = 0.083
R_period_max = 33


N_sims = 500
T1s = zeros(N_sims)
T2s = zeros(N_sims)
R_mins = zeros(N_sims)
R_periods = zeros(N_sims)
Results = zeros(N_sims,dtype=object)
osc_ranges = zeros(N_sims)
has_overflowed = zeros(N_sims, dtype=bool)
alphas = zeros(N_sims)

simnum = 0
np.random.seed(0)
while (simnum < N_sims):
    #Randomly chose dimensionless parameters
    log_T1 = (log_T1_max - log_T1_min)*np.random.rand() + log_T1_min
    log_T2 = (log_T2_max - log_T2_min)*np.random.rand() + log_T2_min
    log_R_period = (log10(R_period_max) - log10(R_period_min))*np.random.rand() + log10(R_period_min)
    T1 = 10.**log_T1
    T2 = 10.**log_T2
    R_min = (R_min_max - R_min_min)*np.random.rand() + R_min_min
    R_period = 10**log_R_period#(R_period_max - R_period_min)*np.random.rand() + R_period_min
    sim = simple_sim(params={'T1':T1,'T2':T2, 'R_func':'sin', 'R_min':R_min, 'R_period':R_period})
    #Check whether sim is stable. Only run stable cases.
    if sim.alpha<0:
        max_step = R_period/50.#0.05
        t_f_min = 50
        min_days=20
        if t_f_min<min_days*R_period:
            t_f = min_days*R_period
        else:
            t_f = t_f_min
                
        print("Running sim number: ",simnum)
        res = sim.run(run_params={'S_0':sim.S_eq, 
                                  'h_0':sim.h_eq,
                                  't_f':t_f,
                                  'Equil_damping_times':1,
                                  'max_step':max_step}
                     )
        if res['status']==0:#Check whether simulation ran correctly
            T1s[simnum] = T1
            T2s[simnum] = T2
            R_mins[simnum] = R_min
            R_periods[simnum] = R_period
            Results[simnum] = res
            alphas[simnum] = sim.alpha
            #Calculate oscilation range near end of sim
            t = sim.sol.t
            t_end = t[-1]
            n_osc_days = 5
            t_start_osc_meas = t_end - n_osc_days*R_period#Last 5 osc periods
            h = sim.sol.y[0]
            h_last = h[t>t_start_osc_meas]#h[t>45]#Runs to 50. 45-50 seems to capture last few oscillations
            osc_range = h_last.max() - h_last.min()
            osc_ranges[simnum] = osc_range
            #Check whether we have the right number of large oscillations in the final period
            h_minus_mean = h_last - mean(h_last)
            n_roots = sum(h_minus_mean[:-1]*h_minus_mean[1:]<0)
            dh_last_dt = h_last[:-1] - h_last[1:]#Check for changes in positive and neg difs
            n_peaks = sum(dh_last_dt[:-1]*dh_last_dt[1:]<0)
            #Check whether water level has gone over floatation
            if h_last.max()>1:
                has_overflowed[simnum] = True
            #Only keep cases with no nans and correct number of roots (random params lead to some unstable cases)
            if not np.isnan(osc_range) and n_peaks==2*n_osc_days:
                ts_filename = './nd-controls-ts/ts-plot-'+str(simnum)+'.png'
                phase_filename = './nd-controls-phase/phase-plot-'+str(simnum)+'.png'
                sim.plot_sim(filename=ts_filename,close_plots=True,tmin=t_start_osc_meas)
                sim.plot_phase(filename=phase_filename, close_plots=True)
                simnum += 1


### Create parameter sensitivity figure (Figure 3)

In [ ]:
figure(figsize=(7.2,6.6))
subplot(2,2,4)
rcParams['font.size']=7

loglog(T1s,osc_ranges,'k.',ms=3)
plot(T1s[has_overflowed],osc_ranges[has_overflowed],'.',color='lightgrey',ms=3)
plot(ex_T1s, ex_osc_ranges, 's',color='orange',ms=5)
xlabel(r'$T_{melt}$')#, fontsize=12)
ylim([0.004,3])
ax = gca()
ax.set_yticklabels([' '])
text(0.95,0.93,'d',fontsize=8, weight='bold',transform=ax.transAxes)

subplot(2,2,3)
loglog(T2s,osc_ranges,'.k',ms=3)
plot(T2s[has_overflowed],osc_ranges[has_overflowed],'.', color='lightgrey',ms=3)
plot(ex_T2s, ex_osc_ranges,  's',color='orange',ms=5)
xlabel(r'$T_{creep}$')#, fontsize=12)
ylabel(r'Moulin head oscilation range ($\Delta h^*$)')#, fontsize=12)
ylim([0.004,3])
ax = gca()
text(0.95,0.93,'c',fontsize=8, weight='bold',transform=ax.transAxes)


subplot(2,2,2)
R_maxs = 2. - R_mins
R_ratios = R_maxs/R_mins
loglog(R_ratios,osc_ranges,'.k',ms=3)
plot(R_ratios[has_overflowed],osc_ranges[has_overflowed],'.',color='lightgrey',ms=3)
plot(ex_R_ratios, ex_osc_ranges,  's',color='orange',ms=5)

rho,p = spearmanr(R_ratios,osc_ranges)
print('rho R_ratios =',rho, '   p=',p)
text(21,0.007, r"$\rho =$"+str(rho)[0:4])#, fontsize=14)
xlabel(r'$R_{max}/R_{min}$')#, fontsize=12)
ylim([0.004,3])
ax = gca()
ax.set_yticklabels([' '])
text(0.95,0.93,'b',fontsize=8, weight='bold',transform=ax.transAxes)



subplot(2,2,1)
loglog(1./R_periods,osc_ranges,'.k', ms=3)
loglog(1./R_periods[has_overflowed],osc_ranges[has_overflowed],'.',color='lightgrey',ms=3)
plot(ex_fs, ex_osc_ranges, 's',color='orange',ms=5)

rho,p = spearmanr(1./R_periods,osc_ranges)
m,b,r,p,err = linregress(log10(1./R_periods)[1./R_periods>0.25], 
                         log10(osc_ranges)[1./R_periods>0.25])
xplot = array([1./0.05,0.25])
yplot = 10**b*xplot**m
plot(xplot,yplot, 'k',lw=2)
plot([0.25,0.25],[0.004,3],'--k')
print('slope=',m)
text(4.,0.75, r"$\rho$ ="+str(rho)[0:4],fontsize=7)
text(3, 1.2, r'$\Delta h^* \propto \tau_{\rm res}^{-0.94}$', fontsize=7)
xlabel(r'$f^* = \tau_{\rm res}/P_{\rm osc}$', fontsize=7)
ylabel(r'Moulin head oscilation range ($\Delta h^*$)', fontsize=7)
ylim([0.004,3])
text(.034,.01, 'small storage',fontsize=7)
ax = gca()
text(0.95,0.93,'a',fontsize=8, weight='bold',transform=ax.transAxes)
tight_layout()
savefig('Sensitivity-composite-fig.png',dpi=300)


### Create small storage sensitivity figure (Figure S1)

In [ ]:
figure(figsize=(3.6,3.3))
small_fs = 1./R_periods<0.25
loglog(R_ratios[small_fs],osc_ranges[small_fs],'.k')
loglog(R_ratios[logical_and(small_fs,has_overflowed)],osc_ranges[logical_and(small_fs,has_overflowed)],'.',color='lightgrey')

rho_small,p_small = spearmanr(R_ratios[small_fs],osc_ranges[small_fs])
print('rho_small=',rho_small)
text(14,0.008*6, r"$\rho =$"+str(rho_small)[0:4], fontsize=7)
xlabel(r'$R_{max}/R_{min}$', fontsize=7)
ylabel(r'Moulin head oscilation range ($\Delta h^*$)',fontsize=7)
tight_layout()
savefig('R_ratio_small_f_osc_range.png',dpi=300)
